In [1]:
import requests
import lxml
from bs4 import BeautifulSoup as bSoup
import pandas as pd

In [2]:
url = 'https://www.amazon.co.uk/gp/bestsellers/computers/430594031/ref=pd_zg_hrsr_computers?fbclid=IwAR2UA1wEfy8AraOyhbecXBgdvxIn6Ki9DikwZRbhmEdhspFp3wpfGO0G_8o'

In [5]:
test = getSoup(url)

In [4]:
def getSoup(url):
    response = requests.get(url)
    responseParsed = bSoup(response.text, 'lxml')
    return responseParsed

In [17]:
def getElementSet(mainSoup, tag, class_):
    products = mainSoup.find_all(tag, class_=class_)
    return products

In [7]:
def cleanText(string):
    string = string.replace('\n',"")
    string = string.replace('\r',"").strip()
    return string

In [93]:
def getElement (parent, tag, class_, isType, extract):
    if parent.find(tag, class_=class_) == None:
        return (None)
    else:
        if isType == '0':
            return cleanText(parent.find(tag, class_=class_).text)
        else:
            return parent.find(tag, class_=class_)[extract]

In [9]:
def getTotalPages(mainSoup):
    pagination = mainSoup.find('ul', class_='a-pagination')
    if pagination.find('li', class_='a-last') == None:
        return (None)
    else:
        lastPage = pagination.find('li', class_='a-last')
        totalPages = lastPage.a['href'][lastPage.a['href'].find('pg'):len(lastPage.a['href'])]
        linkStruct = lastPage.a['href'][0:lastPage.a['href'].find('pg')-1]
        numberPages = int(totalPages[totalPages.find('=')+1:len(totalPages)])
        return linkStruct, numberPages

In [10]:
def makeLinkCollection(mainSoup):
    linkCollection = []
    if getTotalPages(mainSoup) == None:
        return None
    else:
        for i in range(1,getTotalPages(mainSoup)[1]+1):
            linkCollection.append(getTotalPages(mainSoup)[0]+'&pg='+str(i))
        return(linkCollection)

In [67]:
def getElementDict (elementSet, elementDef):
    elementDict = {}
    for element in elementDef:
        elementValue = getElement(elementSet,element['elementTag'],element['elementClass'],element['extractType'],element['elementExtract'])
        elementDict[element['elementName']] = elementValue
    return elementDict

In [95]:
productDef = [
    {'elementName':'Rank','elementTag':'span','elementClass':'zg-badge-text','extractType':'0','elementExtract':'text'},
    {'elementName':'Url','elementTag':'a','elementClass':'a-link-normal','extractType':'1','elementExtract':'href'},
    {'elementName':'Description','elementTag':'div','elementClass':'p13n-sc-truncate','extractType':'0','elementExtract':'text'},
    {'elementName':'Price','elementTag':'span','elementClass':'p13n-sc-price','extractType':'0','elementExtract':'text'},
    {'elementName':'Stars','elementTag':'span','elementClass':'a-icon-alt','extractType':'0','elementExtract':'text'}
]

In [40]:
products = getElementSet(test,'li','zg-item-immersion')

In [108]:
collection = []
for product in products:
    elementDict = getElementDict(product, productDef)
    elementDict['AMZ-Id'] = getLinkSection(elementDict['Url'],'dp/','?')
    collection.append(elementDict)

In [103]:
testingDict

{'Rank': '#1',
 'Url': '/DISDIM-Microphone-Condenser-Recording-Smartphone/dp/B0719CHFLS?_encoding=UTF8&psc=1',
 'Description': 'DISDIM PC Microphone, 3.5mm Jack Condenser Recording Microphone with Mic Stand for PC, Laptop, iPh0ne, iP@d, Mac, Smartphone - Gaming, Singing, YouTube, Skype (Black)',
 'Price': '£20.99',
 'Stars': '3.6 out of 5 stars'}

In [105]:
def getLinkSection(url, start, stop):
        return url[url.find(start)+len(start):url.find(stop)]

In [106]:
print(getLinkSection(testingDict['Url'],'dp/','?'))

B0719CHFLS


In [109]:
collection

[{'Rank': '#1',
  'Url': '/DISDIM-Microphone-Condenser-Recording-Smartphone/dp/B0719CHFLS?_encoding=UTF8&psc=1',
  'Description': 'DISDIM PC Microphone, 3.5mm Jack Condenser Recording Microphone with Mic Stand for PC, Laptop, iPh0ne, iP@d, Mac, Smartphone - Gaming, Singing, YouTube, Skype (Black)',
  'Price': '£20.99',
  'Stars': '3.6 out of 5 stars',
  'AMZ-Id': 'B0719CHFLS'},
 {'Rank': '#2',
  'Url': '/TONOR-Microphone-Condenser-Recording-Podcasting-TC-777/dp/B07W6ZZZWK?_encoding=UTF8&psc=1',
  'Description': 'TONOR PC Microphone USB Computer Condenser Gaming Mic Plug & Play with Tripod Stand & Pop Filter for Vocal Recording, Podcasting, Streaming for iMac PC Laptop Desktop Windows Computer',
  'Price': '£40.99',
  'Stars': '4.6 out of 5 stars',
  'AMZ-Id': 'B07W6ZZZWK'},
 {'Rank': '#3',
  'Url': '/TKGOU-USB-PC-Microphone-Microphones/dp/B07D561S67?_encoding=UTF8&psc=1',
  'Description': 'USB PC Microphone, TKGOU Ture Plug & Play Home Studio USB Computer Microphone With mute button, -

In [110]:
results = pd.DataFrame(collection)

In [111]:
results

,Rank,Url,Description,Price,Stars,AMZ-Id
0,#1,/DISDIM-Microphone-Condenser-Recording-Smartph...,"DISDIM PC Microphone, 3.5mm Jack Condenser Rec...",£20.99,3.6 out of 5 stars,B0719CHFLS
1,#2,/TONOR-Microphone-Condenser-Recording-Podcasti...,TONOR PC Microphone USB Computer Condenser Gam...,£40.99,4.6 out of 5 stars,B07W6ZZZWK
2,#3,/TKGOU-USB-PC-Microphone-Microphones/dp/B07D56...,"USB PC Microphone, TKGOU Ture Plug & Play Home...",£20.39,4.4 out of 5 stars,B07D561S67
3,#4,/Microphone-EIVOTOR-Professional-Condenser-Com...,"USB Microphone for Computer, EIVOTOR Plug & Pl...",£19.98,4.4 out of 5 stars,B07WGF8HG3
4,#5,/Trust-Streaming-Microphone-Connected-Includin...,Trust Gaming GXT 232 Mantis Streaming PC Gamin...,£34.17,4.0 out of 5 stars,B07CZ6LWW4
5,#6,/Mechanical-Keyboard-PICTEK-Anti-ghosting-Cust...,"PICTEK Mechanical Gaming Keyboard, Blue Switch...",£37.99,4.2 out of 5 stars,B07YXXDCMY
6,#7,/Conference-Microphone-Omnidirectional-Microph...,"TONOR Conference USB Microphone, Omnidirection...",£34.99,4.5 out of 5 stars,B07H3KLHNJ
7,#8,/AOBETAK-Microphone-Smartphone-Professional-Co...,AOBETAK External PC Microphone with Stand for ...,£18.89,3.5 out of 5 stars,B07KTTH5S9
8,#9,/Microphone-iGOKU-Omnidirectional-Condenser-In...,"USB Microphone,iGOKU Lavalier Clip-on Omnidire...",£15.99,4.2 out of 5 stars,B076M4HXFH
9,#10,/Blue-Professional-Condenser-Microphone-Podcas...,Blue Yeti x Professional Condenser USB Microph...,£159.98,4.5 out of 5 stars,B07W7KHQLR


In [2]:
productDef = {"elementDefinition":[
    {'elementName':'Rank','elementTag':'span','elementClass':'zg-badge-text','extractType':'0','elementExtract':'text'},
    {'elementName':'Url','elementTag':'a','elementClass':'a-link-normal','extractType':'1','elementExtract':'href'},
    {'elementName':'Description','elementTag':'div','elementClass':'p13n-sc-truncate','extractType':'0','elementExtract':'text'},
    {'elementName':'Price','elementTag':'span','elementClass':'p13n-sc-price','extractType':'0','elementExtract':'text'},
    {'elementName':'Stars','elementTag':'span','elementClass':'a-icon-alt','extractType':'0','elementExtract':'text'}]
}